In [1]:
import sqlalchemy as sa
import os
import sys
import urllib.parse as url
import pandas as pd

In [2]:
# Setup connection parameters
comp = os.environ["COMPUTERNAME"]  # Get the computer name from environment variables
dbase = "Financial_Securities"     # Define the name of the database

username = os.getlogin()
external_folder_path = 'C:/Users/' + username + '/Documents/Projects/Financial_Securities/Custom_Python_Functions'
sys.path.append(external_folder_path)
from custom_python_functions import create_connection, clear_table


# Create a connection to the database
s, e = create_connection(comp, dbase, "", "")
s1 = s()  # Instantiate a session object

In [3]:
Base = sa.orm.declarative_base()

class Sub_Industries(Base):
    
    """
    SQLAlchemy ORM class representing the 'Sub_Industries' table in the 'Equities' schema.

    Attributes:
    __tablename__ (str): The name of the table in the database.
    __table_args__ (dict): Additional arguments for the table, including schema name.
    Sub_Industry_ID (Column): A unique identifier for each sub-industry; primary key.
    Name (Column): The name of the industry.
    Industry_ID (Column): The Industry_ID of the sub-industry.
    """
    
    __tablename__ = 'Sub_Industries'
    __table_args__ = {'schema': 'Equities'}
    Sub_Industry_ID = sa.Column(sa.Integer, primary_key=True, autoincrement=False)
    Name = sa.Column(sa.String)
    Industry_ID = sa.Column(sa.Integer)

In [4]:
# SQL query to select and trim description fields from the Data_STG table
sql_stat = """SELECT 
               Int_Value1, 
               TRIM(Description) AS Description,
			   Int_Value2
              FROM [Financial_Securities].[Equities].[Data_STG]
              ORDER BY Int_Value1"""

try:              
    df_sub_industries = pd.read_sql(sql_stat, s1.bind)   # Execute the SQL query through the session and bind the data to the df_sub_industries dataframe
    
# Handle SQLAlchemy errors if they occur during query execution
except sa.exc.SQLAlchemyError as e:
        # Print the error
        print(f"Issue querying Data_STG database table for count! Error: {e}")
        s1.close()  # Close the session
        raise  # Re-raise the exception to propagate the error
        

In [5]:
for index, row in df_sub_industries.iterrows():
    try:
        
        # Query the 'Industries' table to find records where the 'Industry_ID' column matches the value in the DataFrame's 'Int_Value1' column
        q1 = s1.query(Sub_Industries).filter(Sub_Industries.Sub_Industry_ID == row.Int_Value1)

        # Check if any records were found with the specified 'Sector_ID'
        if (q1.count() >= 1):
            # If one or more records are found, get the first matching record
            q1 = s1.query(Sub_Industries).filter(Sub_Industries.Sub_Industry_ID == row.Int_Value1).first()
            # Update the 'Name' attribute of the found record with the value from the DataFrame's 'Description' column
            q1.Name = row['Description']
            # Update the 'Sub_Industry_ID' attribute of the found record with the value from the DataFrame's 'Int_Value2' column
            q1.Industry_ID = row['Int_Value2']
            
        else:

            # Create a new Industries object for each row in df_industries dataframe
            q1 = Sub_Industries(
                Sub_Industry_ID=row['Int_Value1'],
                Name=row['Description'],
                Industry_ID=row['Int_Value2']
            )
            s1.add(q1)  # Add the instance to the session
    
    # Handle SQLAlchemy errors if they occur during adding the object
    except sa.exc.SQLAlchemyError as e:
        message = f"Issue with updating Sub_Industries database table for Name: {row.Description}. Error: {e}"
        print(message)
        s1.close()  # Close the session
        raise  # Re-raise the exception to propagate the error


s1.commit()  # Commit the transactions to the database

print("Database data load is complete")


In [6]:
# SQL query to count the number of records in the Data_STG table
sql_stat2 = """SELECT COUNT(*) FROM [Financial_Securities].[Equities].[Data_STG]"""
          
try: 
    result1 = e.execute(sql_stat2)  # Execute the count query
    cnt_recs1 = result1.scalar()  # Get the count of records
    
# Handle SQLAlchemy errors if they occur during query execution
except sa.exc.SQLAlchemyError as e:
    print(f"Issue querying Data_STG database table for count! Error: {e}")


# SQL query to count the number of records in the Sub_Industries table
sql_stat3 = """SELECT COUNT(*) FROM [Financial_Securities].[Equities].[Sub_Industries]"""
              
try: 
    result1 = e.execute(sql_stat3)  # Execute the count query
    cnt_recs2 = result1.scalar()  # Get the count of records
    
# Handle SQLAlchemy errors if they occur during query execution
except sa.exc.SQLAlchemyError as e:   
    print(f"Issue querying Sub_Industries database table for count! Error: {e}")

# Compare the record counts and print the result
if cnt_recs2 < cnt_recs1:
    print(f"Only {cnt_recs2} of {cnt_recs1} records were loaded into Sub_Industries database table!")
else:
    print(f"All {cnt_recs2} records were loaded into Sub_Industries database table!") 

All 163 records were loaded into Sub_Industries database table!


In [7]:
s1.close()  # Close the session